In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
batch_size = 12
epochs = 1
model_max_length = 126
label_list = ['O', 'B-c', 'I-c', 'B-p', 'I-p']

## Get data

In [2]:
import dill
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
path_name = "../../dataset/data/"

with open(path_name + 'comment-pos.data', 'rb') as file:
    datatofile = dill.load(file)

tagged_sents = []
for data in datatofile:
    text_inside = []
    for word, pos, label in data:
        if word.strip() == '':
            text_inside.append((' ', pos, label))
        else:
            text_inside.append((word, pos, label))
    tagged_sents.append(text_inside)

train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)
print(len(train_sents))
print(len(test_sents))
print(train_sents[1])

552
138
[('ผญ.', 'NCMN', 'B-c'), (' ', 'PUNC', 'I-c'), ('เวน', 'NCMN', 'I-c'), ('ดี้', 'NCMN', 'I-c'), (' ', 'PUNC', 'I-c'), ('เพราะ', 'JSBR', 'B-p'), ('หน้า', 'NCMN', 'I-p'), ('สวย', 'NCMN', 'I-p'), ('ชอบ', 'VSTA', 'I-p'), ('ผญเเบบ', 'NCMN', 'I-p'), ('เวน', 'NCMN', 'I-p'), ('ดี้', 'NCMN', 'I-p'), ('เสียง', 'NCMN', 'I-p'), ('ใส', 'VATT', 'I-p'), ('พูด', 'ADVN', 'I-p'), ('อิ้ง', 'VACT', 'I-p'), ('เก่ง', 'NCMN', 'I-p'), ('มาก', 'ADVN', 'I-p'), (' ', 'PUNC', 'I-p'), (' ', 'PUNC', 'O'), ('ผช.', 'NCMN', 'B-c'), (' ', 'PUNC', 'I-c'), ('อยาก', 'XVMM', 'I-c'), ('เป็น', 'VSTA', 'I-c'), ('จี', 'NCMN', 'I-c'), ('มิ', 'NEG', 'I-c'), ('น', 'NCMN', 'I-c'), (' ', 'PUNC', 'I-c'), ('อยาก', 'XVMM', 'B-p'), ('เป็น', 'VSTA', 'I-p'), ('หนุ่ม', 'NCMN', 'I-p'), ('เ', 'NCMN', 'I-p'), ('เพ', 'NCMN', 'I-p'), ('รว', 'NCMN', 'I-p'), ('พราว', 'VSTA', 'I-p'), ('ให้', 'JSBR', 'I-p'), ('ผญ.', 'NCMN', 'I-p'), ('ใจ', 'NCMN', 'I-p'), ('ละลาย', 'NCMN', 'I-p'), ('เล่น', 'VACT', 'I-p'), ('ดู', 'VACT', 'I-p'), (' ', 'PUNC',

In [4]:
def data_to_dict(data):
    datasets = dict()
    datasets['id'] = []
    datasets['tokens'] = []
    datasets['pos_tags'] = []
    datasets['ner_tags'] = []
    for i, sent in enumerate(data):
        _word = []
        _label = []
        _pos = []
        for word, pos, label in sent:
            _word.append(word)
            _label.append(label)
            _pos.append(pos)
        datasets['id'].append(i)
        datasets['tokens'].append(_word)
        datasets['pos_tags'].append(_pos)
        datasets['ner_tags'].append([label_list.index(l) for l in _label])
    return datasets

In [5]:
from datasets import DatasetDict, Dataset,Sequence, Features, Value, ClassLabel

ft = Features({
    'id': Value("int32"),
    'tokens': Sequence(Value("string")), 
    'pos_tags': Sequence(Value("string")),
    'ner_tags': Sequence(ClassLabel(names=label_list))
    })

datasets = DatasetDict({
    'train': Dataset.from_dict(data_to_dict(train_sents), features=ft),
    'test': Dataset.from_dict(data_to_dict(test_sents), features=ft)
})

datasets

2022-04-21 15:02:27.744582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-21 15:02:27.744640: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 552
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 138
    })
})

In [6]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
print('len:', len(label_list))
print(label_list)

len: 5
['O', 'B-c', 'I-c', 'B-p', 'I-p']


In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

,id,tokens,pos_tags,ner_tags
0,339,"[ชอบ, ทำงาน, ที่, ออฟ, ฟิส, , คอม, หน้าจอ, ใหญ่, กว่า, , ป, ริ้น, งาน, มา, นั่ง, ดู, ได้, ด้วย, , คุย, กับ, ทีม, ง่าย, กว่า, นั่ง, แชท, , สรุป, งาน, เสร็จ, เร็ว, กว่า, , ดีกว่า, เน็ต, กาก, ๆ, ที่, บ้าน, , แถม, โต๊ะ, กับ, เก้าอี้นั่ง, แล้ว, ไม่, ปวดหลัง, , (, จริงๆ, ก็, อยาก, ซื้อ, เก้าอี้, ดี, ๆ, นะ, , แต่, ต้อง, ประหยัด, ตัง, )​, เลิกงาน, แล้วก็, เลิกงาน, จริง, , wfh, , บางที, มี, คน, โทร, หา, ทั้งที่, เรา, เลิกงาน, แล้ว, , รู้สึก, เหมือน, เวลา, ส่วนตัว, กับ, เวลา, งาน, กลืนกัน, ไป, หมด]","[VSTA, VACT, RPRE, NCMN, NCMN, PUNC, NCMN, NCMN, VATT, JCMP, PUNC, NCMN, NCMN, NCMN, XVAE, VACT, VACT, XVAE, RPRE, PUNC, NCMN, RPRE, NCMN, VATT, JCMP, NCMN, DCNM, PUNC, VACT, NCMN, VSTA, ADVN, JCMP, PUNC, JSBR, NCMN, NCMN, NCMN, PREL, NCMN, PUNC, NCMN, NCMN, RPRE, NCMN, XVAE, NEG, VSTA, PUNC, PUNC, ADVI, JSBR, NCMN, VACT, NCMN, VATT, PUNC, NCMN, PUNC, JCRG, XVMM, VACT, NCMN, NCMN, NCMN, JCRG, NCMN, VATT, PUNC, NCMN, PUNC, NCMN, VSTA, NCMN, NCMN, VACT, PREL, PPRS, VACT, XVAE, PUNC, NCMN, VSTA, NCMN, VSTA, RPRE, NCMN, NCMN, VSTA, XVAE, ADVN]","[B-c, I-c, I-c, I-c, I-c, O, B-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,492,"[ภาค, ตะวันออก, , , เพราะ, มี, พี่, อยู่, จว, , ตราด, , หน้าบ้าน, เป็น, ถนน, ใน, ซอยตัน, ลึก, 200, , เมตร, เท่านั้น, , หลังบ้าน, เป็น, แม่น้ำ, ตราด, , อยู่, ไกล, จาก, ตลาด, ใน, ตัว, จังหวัด, 2, , กม., , สงบ, , อาหาร, อุดม, สม, บูร, ์, ทั้ง, ทะเล, จาก, อ่าวไทย, , พืช, ผลไม้, จาก, สวน, , ตลอดจน, เกาะ, , น้ำตก, , อากาศ, ไม่, ร้อน, ตับ, แตก, แบบ, เหนือ, คห., , ข้างบน, บอก, , และ, ไม่, ฝนตก, ทั้งปี, แบบ, ภาคใต้, , มี, สาม, ฤดู, , หน้าหนาว, มี, , หน้าฝน, ก็, ใช่, , ชาว, สวนผลไม้, มี, เจอ, แล้ง, บาง, ปี, ]","[NCMN, NCMN, PUNC, PUNC, JSBR, VSTA, NCMN, XVAE, NCMN, PUNC, NCMN, PUNC, NCMN, VSTA, NCMN, RPRE, NCMN, VATT, DCNM, PUNC, CMTR, ADVN, PUNC, NCMN, VSTA, NCMN, NCMN, NCMN, VSTA, ADVN, RPRE, NCMN, RPRE, NCMN, NCMN, DCNM, PUNC, CMTR, PUNC, NCMN, PUNC, NCMN, NCMN, NCMN, NCMN, NCMN, DDBQ, NCMN, RPRE, NPRP, PUNC, NCMN, NCMN, RPRE, NCMN, PUNC, JCRG, NCMN, PUNC, NCMN, NCMN, NCMN, NEG, VATT, NCMN, NCMN, NCMN, RPRE, NCMN, PUNC, NCMN, VACT, PUNC, JCRG, NEG, VACT, NCMN, NCMN, NCMN, PUNC, VSTA, DCNM, NCMN, PUNC, NCMN, VSTA, PUNC, NCMN, JSBR, VSTA, PUNC, NCMN, NCMN, VSTA, NCMN, NCMN, DIBQ, CMTR, PUNC]","[B-c, I-c, I-c, O, B-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p]"
2,25,"[ตอบ, ได้, คำ, เดียว, สั้น, ๆ, ง่ายๆ, , จริง, แท้, แน่นอน, คะ, , ไม่, ควร, ให้, ลูก, ดู, ทีวี, , เล่น, ไอ, เพ, ด, , จนกว่า, อายุ, จะ, ครบ, 2, ขวบ, , มี, งานวิจัย, ออกมา, แล้ว, มากมาย, คะ, , ว่าการ, ดู, ทีวี, จะ, ทำให้, เด็ก, สมาธิ, สั้น, จิงๆ, ค่า]","[VACT, XVAE, NCMN, DCNM, JSBR, PUNC, NCMN, PUNC, VATT, CNIT, VATT, NCMN, PUNC, NEG, XVMM, VACT, NCMN, VACT, NCMN, PUNC, VACT, NCMN, NCMN, NCMN, PUNC, JSBR, NCMN, XVBM, VSTA, NCNM, NCMN, PUNC, VSTA, NCMN, XVAE, XVAE, ADVN, NCMN, PUNC, NCMN, VACT, NCMN, XVBM, VACT, NCMN, NCMN, VATT, VACT, NCMN]","[B-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, O, B-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p]"
3,76,"[ถ้า, คุณ, ​, เรียนจบ, หา, แนวทาง, ทำงาน, ต่างประเทศ, ​, ได้, ถือว่า, คุ้ม, มาก, ๆ, ค่ะ, ​, , วิศวะ, ถือ, เป็นงาน, , special, ​, ist, , ใน, หลา

## Preprocess The data

In [8]:
from transformers import AutoTokenizer  
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=model_max_length)

In [9]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [10]:
example = datasets["train"][2]
print(example["tokens"])

['เครื่องกล', ' ', ' ', 'เพราะ', 'หน้า', 'งาน', 'กว้าง', 'มาก', 'ๆ', ' ']


In [11]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁', 'เครื่อง', 'กล', '▁', '▁', '▁เพราะ', '▁', 'หน้า', '▁งาน', '▁', 'กว้าง', '▁', 'มาก', '▁', 'ๆ', '▁', '</s>']


In [12]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

18 18


In [13]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
tokenize_and_align_labels(datasets['train'][:1])

{'input_ids': [[5, 391, 10, 91, 10, 961, 10, 10, 28, 10, 10, 81, 10, 3742, 3325, 10, 10, 16820, 10, 10, 5261, 10, 158, 10, 10, 10, 2817, 10, 2732, 10, 158, 10, 222, 10, 2817, 10, 822, 10, 158, 10, 10, 391, 1896, 10, 200, 10, 593, 10, 9396, 10, 28, 10, 10, 3, 10569, 1189, 10, 3310, 10, 3038, 10, 12677, 5678, 10, 294, 10, 1905, 10, 73, 6]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [15]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)
tokenized_datasets

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 552
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 138
    })
})

## Fine-tuning the model

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier

In [17]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= epochs,
    weight_decay=0.01,
    save_strategy='epoch',
    save_total_limit=2,
)

In [18]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [19]:
from datasets import load_metric
metric = load_metric("seqeval")

In [20]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'c': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'p': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [21]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [22]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
os.environ["WANDB_DISABLED"] = "true"

In [24]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
/home/mike/project/env_project_argument/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 552
  Num Epochs = 1
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 46
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Faile

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2022-04-21 15:02:55.392160: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-21 15:02:55.392212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.084610,0.012270,0.025806,0.016632,0.549348


The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12
Saving model checkpoint to wangchanberta-base-att-spm-uncased-finetuned-ner/checkpoint-46
Configuration saved in wangchanberta-base-att-spm-uncased-finetuned-ner/checkpoint-46/config.json
Model weights saved in wangchanberta-base-att-spm-uncased-finetuned-ner/checkpoint-46/pytorch_model.bin
tokenizer config file saved in wangchanberta-base-att-spm-uncased-finetuned-ner/checkpoint-46/tokenizer_config.json
Special tokens file saved in wangchanberta-base-att-spm-uncased-finetuned-ner/checkpoint-46/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=46, training_loss=1.1841389199961787, metrics={'train_runtime': 42.1679, 'train_samples_per_second': 13.091, 'train_steps_per_second': 1.091, 'total_flos': 35496494036640.0, 'train_loss': 1.1841389199961787, 'epoch': 1.0})

In [25]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12


{'eval_loss': 1.0846095085144043,
 'eval_precision': 0.012269938650306749,
 'eval_recall': 0.025806451612903226,
 'eval_f1': 0.016632016632016633,
 'eval_accuracy': 0.5493477027793534,
 'eval_runtime': 2.4427,
 'eval_samples_per_second': 56.495,
 'eval_steps_per_second': 4.913,
 'epoch': 1.0}

In [26]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 138
  Batch size = 12


{'c': {'precision': 0.02512562814070352,
  'recall': 0.0364963503649635,
  'f1': 0.029761904761904764,
  'number': 274},
 'p': {'precision': 0.0034482758620689655,
  'recall': 0.010471204188481676,
  'f1': 0.005188067444876784,
  'number': 191},
 'overall_precision': 0.012269938650306749,
 'overall_recall': 0.025806451612903226,
 'overall_f1': 0.016632016632016633,
 'overall_accuracy': 0.5493477027793534}

In [27]:
idx = 20
show_text = tokenizer(tokenized_datasets["test"][idx]['tokens'], is_split_into_words=True)
print(tokenizer.convert_ids_to_tokens(show_text['input_ids']))

['<s>', '▁นี่', '▁', 'ไง', '▁', '▁ใคร', '▁', 'บอ', '▁', 'กว่า', '▁', 'ชื่อเสียง', '▁', 'มหาวิทยาลั', 'ย', '▁ไม่', '▁', 'สําคัญ', '▁', '▁สําหรับ', '▁เด็ก', '▁', 'จบ', '▁', 'ใหม่', '▁', '▁', 'สําคัญ', '▁', 'มาก', '▁', 'ครับ', '▁', '▁เคย', '▁', 'ทํางาน', '▁บริษัท', '▁', 'จัดหา', '▁งาน', '▁', '▁', 'ลูกค้า', '▁ขอ', '▁', 'มา', '▁', 'เลย', '▁', '▁', 'สายงาน', '▁', 'นี้', '▁ต้อง', '▁', 'มหาลัย', '▁', 'นี้', '▁', 'เท่านั้น', '▁', '▁เพราะ', '▁คุณ', '▁ไม่', '▁มี', 'ประสบการณ์', '▁', '▁', 'เลย', '▁ต้อง', '▁ใช้', '▁', 'มหาวิทยาลั', 'ย', '▁', 'มา', '▁', 'เป็น', '▁', 'เกณฑ์', '▁', 'ใน', '▁การ', '▁', 'ตัดสิน', '▁', '▁', 'อย่าง', '▁', 'น้อย', '▁', 'มหาวิทยาลั', 'ย', '▁', 'ที่', '▁มี', 'ชื่อเสียง', '▁', '▁ก็', '▁', 'วัด', '▁', 'ได้', '▁', 'ประมาณ', '▁', 'หนึ่ง', '</s>']


In [28]:
print('pred:', true_predictions[idx])
print('true:', true_labels[idx])

pred: ['I-c', 'I-c', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-c', 'I-c', 'I-c', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-c']
true: ['B-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'O', 'B-p', 'I-p', 'I-p', '

## Predict Text

In [29]:
from pythainlp.tokenize import word_tokenize
text_list = word_tokenize('ฉันชอบหมาเพราะมันน่ารักมากๆ เลย')
print(text_list)

['ฉัน', 'ชอบ', 'หมา', 'เพราะ', 'มัน', 'น่ารัก', 'มาก', 'ๆ', ' ', 'เลย']


In [30]:
_input_token = tokenizer(text_list, is_split_into_words=True)
_word_token = tokenizer.convert_ids_to_tokens(_input_token["input_ids"])
print(_input_token)
print(_word_token)

{'input_ids': [5, 264, 1879, 10, 1022, 474, 661, 5840, 10, 82, 10, 34, 10, 10, 48, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['<s>', '▁ฉัน', '▁ชอบ', '▁', 'หมา', '▁เพราะ', '▁มัน', '▁น่ารัก', '▁', 'มาก', '▁', 'ๆ', '▁', '▁', 'เลย', '</s>']


In [31]:
pred = trainer.predict([_input_token])[0]
pred = np.argmax(pred, axis=2)
pred

***** Running Prediction *****
  Num examples = 1
  Batch size = 12


array([[4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]])

In [32]:
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(pred, labels)
][0]
print(true_predictions)

['I-c', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p']


In [33]:
print(len(true_predictions))
print(len(_word_token))

15
16


In [34]:
for w, l in zip(_word_token[1:-1], true_predictions):
  print((w, l))

('▁ฉัน', 'I-c')
('▁ชอบ', 'I-p')
('▁', 'I-p')
('หมา', 'I-p')
('▁เพราะ', 'I-p')
('▁มัน', 'I-p')
('▁น่ารัก', 'I-p')
('▁', 'I-p')
('มาก', 'I-p')
('▁', 'I-p')
('ๆ', 'I-p')
('▁', 'I-p')
('▁', 'I-p')
('เลย', 'I-p')
